In [ ]:
import tensorflow as tf
import numpy as np
import PIL
import os

In [ ]:
def load_image(img_file, target_size=(224,224)):
    X = np.zeros((1, *target_size, 3))
    X[0, ] = np.asarray(tf.keras.preprocessing.image.load_img(
        img_file, 
        target_size=target_size)
    )
    X = tf.keras.applications.mobilenet.preprocess_input(X)
    return X

def ensure_folder_exists(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet()
model.summary()

In [ ]:
model = tf.keras.applications.mobilenet.MobileNet(
  input_shape=(224, 224, 3), 
  include_top=False, 
  pooling='avg'
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dropout, Dense, Softmax)

x = Dropout(rate=0.4)(model.output)
x = Dense(3)(x)
x = Softmax()(x)
model= Model(model.inputs, x)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers[:-3]:
    layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(lr=0.001),
    loss='categorical_crossentropy'
)

In [ ]:
from tensorflow.keras.applications import mobilenet as _mobilenet

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=_mobilenet.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

In [ ]:
ensure_folder_exists('training_aug')
training = datagen.flow_from_directory( 
    './images', 
    target_size=(224, 224),
    save_to_dir='./training_aug',
    subset='training'
) 

ensure_folder_exists('validation_aug')
validation = datagen.flow_from_directory( 
    './images',
    target_size=(224, 224),
    save_to_dir='./validation_aug',
    subset='validation'
) 

In [ ]:
from keras_tqdm import TQDMNotebookCallback

batch_size = 32

history = model.fit_generator(
    generator=training,
    steps_per_epoch=training.samples // batch_size,
    epochs=10,
    callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)],
    validation_data=validation,
    validation_steps=validation.samples // batch_size
)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
murray_test = load_image('murray.png')
mortez_test = load_image('mortez.png')
mortez1_test = load_image('mortez1.png')
false_test = load_image('false.png')
pfeiffer_test = load_image('pfeiffer.png')

In [ ]:
from numpy import random

random_mortez = random.choice(os.listdir('images/chloë_grace_moretz/'))
mortez_path = os.path.join('images/chloë_grace_moretz',random_mortez)
mortez2_test = load_image(mortez_path)

random_murray = random.choice(os.listdir('images/f._murray_abraham/'))
murray_path = os.path.join('images/f._murray_abraham',random_murray)
murray1_test = load_image(murray_path)

random_pfeiffer = random.choice(os.listdir('images/michelle_pfeiffer/'))
pfeiffer_path = os.path.join('images/michelle_pfeiffer',random_pfeiffer)
pfeiffer1_test = load_image(pfeiffer_path)

In [ ]:
display(tf.keras.preprocessing.image.load_img('murray.png', target_size=(224,224)))
print(f'''murray: {np.round(model.predict(murray_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img(murray_path, target_size=(224,224)))
print(f'''mortez1: {np.round(model.predict(murray1_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img('mortez.png', target_size=(224,224)))
print(f'''mortez: {np.round(model.predict(mortez_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img('mortez1.png', target_size=(224,224)))
print(f'''mortez1: {np.round(model.predict(mortez1_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img(mortez_path, target_size=(224,224)))
print(f'''mortez2: {np.round(model.predict(mortez2_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img('false.png', target_size=(224,224)))
print(f'''false: {np.round(model.predict(false_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img('pfeiffer.png', target_size=(224,224)))
print(f'''pfeiffer: {np.round(model.predict(pfeiffer_test), 2)
}''')

In [ ]:
display(tf.keras.preprocessing.image.load_img(pfeiffer_path, target_size=(224,224)))
print(f'''pfeiffer: {np.round(model.predict(pfeiffer1_test), 2)
}''')